In [6]:
!pip install deepseek-vl

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os

os.chdir(r"/nfs/home/tgv3756/dlproject/scripts/deepseek_vl/DeepSeek-VL2")

In [2]:
import torch
from transformers import AutoModelForCausalLM
import torch.multiprocessing as mp

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images

In [3]:
torch.cuda.set_device(4)

In [4]:
mp.set_start_method('spawn')

In [5]:
# Loading DS VL2 tiny model

model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

/nfs/home/tgv3756/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



In [6]:
vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [150]:
# conversation = [
#     {
#         "role": "<|User|>",
#         "content": "<image>\n<|ref|>The driver is.<|/ref|>.",
#         "images": ["/nfs/home/tgv3756/dlproject/data/imgs/train/c4/img_440.jpg"],
#     },
#     {"role": "<|Assistant|>", "content": ""},
# ]



conversation = [
    {
        "role": "<|User|>",
        "content": "<image>",
        "images": ["/nfs/home/tgv3756/dlproject/data/imgs/train/c6/img_678.jpg"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

In [151]:
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

In [152]:
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

In [153]:
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512, ##512,
    do_sample=True,
    use_cache=True
)

In [154]:
answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True) ##=False)

In [155]:
answer

"The man is drinking from his own water bottle while sitting in the driver's seat of his car. He appears calm but could be distracted by something happening outside or within the vehicle at that moment. The presence of a steering wheel indicates he has control over the vehicle currently. There doesn't seem to be any specific event occurring based solely on this snapshot; it looks like an ordinary scene where someone might drink during their commute or leisure time. It’s important for drivers to stay aware of traffic conditions and other road users when drinking beverages behind the wheel."

In [24]:
print(f"{prepare_inputs['sft_format'][0]}", answer)

<|User|>: <image>
<|ref|>The driver is.<|/ref|>.

<|Assistant|>: <|ref|>The driver is.<|/ref|><|det|>[[0, 0, 999, 999]]<|/det|><｜end▁of▁sentence｜>
